<a href="https://colab.research.google.com/github/marevalojaimes/Arevalo_DSPN_S25/blob/main/Exercise10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 10: Mixed effects

1. Loading and formatting the data 1/1
2. Model fitting 4/4
3. Model assessment 4/4
4. Reflection 1/1

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:
library(readr)
library(dplyr)
library(tidyr)
lexical_inc_data <- read_csv("https://raw.githubusercontent.com/marevalojaimes/Data_ScienceCMU/refs/heads/master/Homework%20datasets/lexDat/LexicalData_withIncorrect.csv")
items_data <- read_csv ("https://raw.githubusercontent.com/marevalojaimes/Data_ScienceCMU/refs/heads/master/Homework%20datasets/lexDat/Items.csv")
lexical_inc_data$D_RT<- as.numeric(gsub("," , "", lexical_inc_data$D_RT))

items <- items_data %>%
  select (Word,Length,Log_Freq_HAL) %>%
  rename(D_Word = Word)

Lexical_merged <- lexical_inc_data %>%
  left_join(items, by = "D_Word") %>%
  drop_na()

head (Lexical_merged)

New names:
• `` -> `...1`
Rows: 74869 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): D_Word
dbl (6): ...1, Sub_ID, Trial, Type, D_Zscore, Correct
num (1): D_RT
lgl (1): Outlier

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 30959 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Word
dbl (3): Occurrences, Length, Log_Freq_HAL
num (1): Freq_HAL

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,synergistic,11,284,5.649
1,synonymous,10,951,6.858
1,syntactical,11,114,4.736
1,synthesis,9,6742,8.816
1,synthesized,11,2709,7.904
1,synthesizer,11,1390,7.237


...1,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Correct,Length,Log_Freq_HAL
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>
1,157,1,1,710,browse,FALSE,-0.437,1,6,8.856
2,67,1,1,1094,refrigerant,FALSE,0.825,1,11,4.644
3,120,1,1,587,gaining,FALSE,-0.645,1,7,8.304
4,21,1,1,984,cheerless,FALSE,0.025,1,9,2.639
5,236,1,1,577,pattered,FALSE,-0.763,1,8,1.386
6,236,2,1,715,conjures,FALSE,-0.364,1,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
model1 =lm(D_RT ~ Log_Freq_HAL * Length  , data = Lexical_merged)
summary(model1)




Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = Lexical_merged)

Residuals:
    Min      1Q  Median      3Q     Max 
-1128.3  -217.6   -94.0    94.2  3317.2 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         650.3764    14.3247  45.403  < 2e-16 ***
Log_Freq_HAL        -10.0802     1.9643  -5.132 2.88e-07 ***
Length               45.5806     1.5992  28.503  < 2e-16 ***
Log_Freq_HAL:Length  -2.6346     0.2345 -11.236  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 384.2 on 70585 degrees of freedom
Multiple R-squared:  0.08867,	Adjusted R-squared:  0.08863 
F-statistic:  2289 on 3 and 70585 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
install.packages("lme4")
library(lme4)



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rdpack’, ‘reformulas’, ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
model2<-lmer(D_RT ~ Log_Freq_HAL * Length + (1| Sub_ID)  , data = Lexical_merged)
summary(model2)



Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: Lexical_merged

REML criterion at convergence: 1012299

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2579 -0.5401 -0.1555  0.2986 11.0922 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 51061    226.0   
 Residual             97009    311.5   
Number of obs: 70589, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         652.6262    17.4987  37.296
Log_Freq_HAL        -11.0297     1.5960  -6.911
Length               45.7392     1.2990  35.211
Log_Freq_HAL:Length  -2.5742     0.1905 -13.514

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.621                
Length      -0.635  0.913         
Lg_Fr_HAL:L  0.561 -0.943   -0.919

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> The t-value of Log_Freq_Hal in the linear model (-5.13) is smaller compared to the mixed effect model (-6.91). Similarly, length follow this trend with a greater t-value in the mixed effects model (35.21) than in the fixed effects (28.50). Lastly, the interaction between lenght and frequency showed also a larger effect in the mixed model (-13.51) in contrast to the linear model(-11.23).   

We can conclude that the mixed effect model has an overall better fit than the lineal model because it accounts for the subject variability, which allow us to make more accurate predictions of the outcome with our variables word lengh and frequency.

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better?

In [ ]:
aic_lm <- AIC(model1)
aic_mx <- AIC(model2)
print(aic_lm)
print(aic_mx)
print(aic_lm-aic_mx)

[1] 1040519
[1] 1012311
[1] 28208.17


> There is a large difference between linear and mixed models (28208.17). Particularly, the mixed model has lower AIC which indicates, this model is better than the linear model.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?

> It would be important to control for whether the RT is affected by random effects of correct/incorrect responses. Also, I think trial type might be useful to explore because the effects of word lenght and frequency could vary across trials.
>

**DUE:** 5pm EST, March 18, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*